# Getting and preparing the data



To demonstrate the post processing algorithm we use the "Adult Data Set" from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/adult). The task there is to predict whether a person makes more or less than $50k based on just a few attributes such as age, sex, race, education, occupation, etc.

To start, let's download the dataset:

In [2]:
import pandas as pd
import numpy as np

columns = ['age', 'workclass', 'fnlwgt', 'education_text', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

data = pd.read_csv('adult.data', names=columns)
data_test = pd.read_csv('adult.test', names=columns, skiprows=[0])

data.head()

,age,workclass,fnlwgt,education_text,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Since we need to encode categorical attributes properly, there are a few additional steps to take.

In [3]:
# one hot encoding
def transform_data(data):
    X = data[['age', 'education', 'capital_gain', 'capital_loss', 'hours_per_week']]
    X = X.join((data.sex == " Male") * 1)
    X = X.join(pd.get_dummies(data.workclass))
    del X[' ?']
    X = X.join(pd.get_dummies(data.occupation))
    del X[' ?']
    X = X.join(pd.get_dummies(data.relationship))
    X = X.join(pd.get_dummies(data.race))
    X = X.join(pd.get_dummies(data.native_country))
    y = ((data.income == ' >50K') | (data.income == ' >50K.')) * 1
    return X, y

full_X, full_y = transform_data(pd.concat([data, data_test], ignore_index=True))
X_train, y_train = full_X[:len(data)], full_y[:len(data)]
X_test, y_test = full_X[len(data):], full_y[len(data):]
X_test = X_test.set_index(pd.Index(list(range(len(X_test)))))
y_test.index = pd.Index(list(range(len(y_test))))
X_train.join(y_train).head()

,age,education,capital_gain,capital_loss,hours_per_week,sex,Federal-gov,Local-gov,Never-worked,Private,...,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia,income
0,39,13,2174,0,40,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,50,13,0,0,13,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,38,9,0,0,40,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,53,7,0,0,40,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,28,13,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


# Create a fairness-unaware model

In [4]:
from sklearn.linear_model import LogisticRegression

fairness_unaware_model = LogisticRegression()
fairness_unaware_model.fit(X_train, y_train)

def show_proportions(X, y, description):
    male_indices = X.index[X.sex == 1]
    female_indices = X.index[X.sex == 0]

    males_earning_more_than_50k = sum(y[male_indices])
    females_earning_more_than_50k = sum(y[female_indices])
    print(description)
    print("P[income > 50k | male] = {}".format(males_earning_more_than_50k/len(male_indices)))
    print("P[income > 50k | female] = {}".format(females_earning_more_than_50k/len(female_indices)))
    
show_proportions(X_train, y_train, "original training data:")
show_proportions(X_train, fairness_unaware_model.predict(X_train), "fairness-unaware prediction on training data:")
show_proportions(X_test, y_test, "original test data:")
show_proportions(X_test, fairness_unaware_model.predict(X_test), "fairness-unaware prediction on test data:")

c:\users\rolutz\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

# Post-processing the model to get a fair model

The idea behind post-processing is to alter the output of the fairness-unaware model to achieve fairness. The post-processing algorithm requires three input arguments:
- the matrix of samples X
- the vector of predictions y from the fairness-unaware model 
- the vector of protected attribute values A

The goal is to make the output fair with respect to a fairness metric. Our post-processing algorithm uses one of
- Demographic Parity (DP): $P[h(X)=\hat{y} | A=a] = P[h(X)=\hat{y}] \qquad \forall a, \hat{y}$
- Equalized Odds (EO): $P[h(X)=\hat{y} | A=a, Y=y] = P[h(X)=\hat{y}|Y=y] \qquad \forall a, \hat{y}$

where $h(X)$ is the prediction based on the input $X$, $\hat{y}$ and $y$ are labels, and $a$ is a protected attribute value. In our example, we'd expect the post-processed model with DP to be balanced between sexes. EO does not make the same guarantees. Instead, it ensures that the parity between the subgroups of each sex with label 1 in the training set, and parity between the subgroups of each sex with label 0 in the training set. Applied to our scenario, this means that men and women with income over $\$50,000$ are equally likely to have a predicted income over $\$50,000$ (and therefore also equally likely to have a predicted income no higher than $\$50,000$). Similarly, there is parity between men and women with income under $\$50,000$, but we have no parity between the groups with different training labels. In mathematical terms:

$$
P[\text{predicted income > 50k} | \text{male, income > 50k}] = P[\text{predicted income > 50k} | \text{female, income > 50k}], \text{e.g. } 0.95\\
P[\text{predicted income > 50k} | \text{male, income <= 50k}] = P[\text{predicted income > 50k} | \text{female, income <= 50k}], \text{e.g. } 0.15
$$

but that also means that men (and women) of different subgroup based on training labels don't necessarily have parity:

$$
P[\text{predicted income > 50k} | \text{male, income > 50k}] = 0.95 <> 0.15 = P[\text{predicted income <= 50k} | \text{male, income <= 50k}]
$$

Assessing which fairness metric is indeed fair varies by application scenario. TODO: link to papers that criticize fairness metrics

In [1]:
from fairlearn.post_processing.roc_curve_based_post_processing import roc_curve_based_post_processing_demographic_parity, roc_curve_based_post_processing_equalized_odds

post_processed_model_DP = roc_curve_based_post_processing_demographic_parity(X_train.sex.values, y_train.values, fairness_unaware_model.predict(X_train), plot=True)
post_processed_model_EO = roc_curve_based_post_processing_equalized_odds(X_train.sex.values, y_train.values, fairness_unaware_model.predict(X_train), plot=True)

fairness_unaware_predictions_train = fairness_unaware_model.predict(X_train)
fairness_unaware_predictions_test = fairness_unaware_model.predict(X_test)

fairness_aware_predictions_DP = post_processed_model_DP(X_train.sex, fairness_unaware_predictions)
fairness_aware_predictions_EO = post_processed_model_EO(X_train.sex, fairness_unaware_predictions)

show_proportions(X_train, pd.Series(post_processed_model_DP(X_train.sex, fairness_unaware_predictions_train)), "demographic parity with post-processed model on training data:")
show_proportions(X_test, pd.Series(post_processed_model_DP(X_test.sex, fairness_unaware_predictions_test)), "demographic parity with post-processed model on test data:")

show_proportions(X_train, pd.Series(post_processed_model_EO(X_train.sex, fairness_unaware_predictions_train)), "equalized odds with post-processed model on training data:")
show_proportions(X_test, pd.Series(post_processed_model_EO(X_test.sex, fairness_unaware_predictions_test)), "equalized odds with post-processed model on test data:")

NameError: name 'X_train' is not defined